In [2]:
counters = {
    'pumpernickel': 2,
    'sourdough': 1,
}
key = 'wheat'

# 使用in来判断key是否存在
if key in counters:
    counters[key] += 1
else:
    counters[key] = 1

print(counters)

# 使用try/except来处理key不存在的问题
key = 'brioche'

try:
    counters[key] += 1
except KeyError:
    counters[key] = 1

print(counters)

{'pumpernickel': 2, 'sourdough': 1, 'wheat': 1}
{'pumpernickel': 2, 'sourdough': 1, 'wheat': 1, 'brioche': 1}


In [3]:
# 前两者没法消除重复赋值
# 如果字典的value是比较简单的类型，优先考虑get方法
key = 'multigrain'

count = counters.get(key, 0)
counters[key] = count + 1

print(counters)

{'pumpernickel': 2, 'sourdough': 1, 'wheat': 1, 'brioche': 1, 'multigrain': 1}


In [7]:
# 字典的value是list
votes = {
    'baguette': ['Bob', 'Alice'],
    'ciabatta': ['Coco', 'Deb'],
}
key = 'brioche'
who = 'Elmer'

# 使用in判断key是否存在
if key in votes:
    names = votes[key] # names绑定字典的value，也就是绑定一个list
else:
    votes[key] = names = [] # names绑定一个空list

names.append(who) # 往list里添加元素
print(votes)

# 使用try/except判断key是否存在
key = 'rye'
who = 'Felix'

try:
    names = votes[key]
except KeyError:
    votes[key] = names = []

names.append(who)

print(votes)

# 使用get处理key不存在问题
key = 'wheat'
who = 'Gertrude'

# 只有字典的value是简单值时用get是最简洁的
names = votes.get(key) # 只能返回不能绑定，所以需要绑定一下
if names is None: 
    votes[key] = names = []

names.append(who)

print(votes)

# get加上赋值表达式
key = 'brioche'
who = 'Hugh'

if (names := votes.get(key)) is None:
    votes[key] = names = []

names.append(who)

print(votes)

{'baguette': ['Bob', 'Alice'], 'ciabatta': ['Coco', 'Deb'], 'brioche': ['Elmer']}
{'baguette': ['Bob', 'Alice'], 'ciabatta': ['Coco', 'Deb'], 'brioche': ['Elmer'], 'rye': ['Felix']}
{'baguette': ['Bob', 'Alice'], 'ciabatta': ['Coco', 'Deb'], 'brioche': ['Elmer'], 'rye': ['Felix'], 'wheat': ['Gertrude']}
{'baguette': ['Bob', 'Alice'], 'ciabatta': ['Coco', 'Deb'], 'brioche': ['Elmer', 'Hugh'], 'rye': ['Felix'], 'wheat': ['Gertrude']}


In [15]:
# 回顾一下
visits = {
    'Mexico': {'Tulum', 'Puerto Vallarta'},
    'Japan': {'Hakone'},
}
if (japan := visits.get('Japan')) is None:       # Long
    visits['Japan'] = japan = set()
japan.add('Kyoto')
original_print = print
from pprint import pprint
print = pprint # pprint可以输出复杂对象的编译器源码

print(visits,sort_dicts=False) # pprint的sort_dects默认开启，按照key排序输出
print = original_print # 还原原来的print

{'Mexico': {'Puerto Vallarta', 'Tulum'}, 'Japan': {'Kyoto', 'Hakone'}}


In [19]:
# 复杂字典直接封装成类，使用collections中的defaultdict来实现key缺失
# 也就是字典需要添加任意的key时，使用defaultdict
from collections import defaultdict

class Visits:
    def __init__(self):
        self.data = defaultdict(set)

    def add(self, country, city):
        self.data[country].add(city) # 如果key不存在，会创建key并添加空set，然后往set中添加元素

visits = Visits()
visits.add('England', 'Bath')
visits.add('England', 'London')
print(visits.data)

defaultdict(<class 'set'>, {'England': {'Bath', 'London'}})


In [22]:
# 临时的输出文件
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# 确保窗口结束清理生成的文件
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

In [27]:
# 字典里的图片路径和相关的文件句柄关联，方便读取并写入图片

# 使用get方法
import os
os.getcwd()

pictures = {}
path = 'profile_1234.png'

with open(path, 'wb') as f: # 写点数据进去
    f.write(b'image data here 1234')

if (handle := pictures.get(path)) is None:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()

print(pictures) # pictures里存的是文件句柄，可以直接读写
print(image_data)

{'profile_1234.png': <_io.BufferedRandom name='profile_1234.png'>}
b'image data here 1234'


In [25]:
# 使用defaultdict报错，因为defaultdict只能传不带参数的函数
import logging
try:
    path = 'profile_4555.csv'
    
    with open(path, 'wb') as f:
        f.write(b'image data here 9239')
    
    from collections import defaultdict
    
    def open_picture(profile_path):
        try:
            return open(profile_path, 'a+b')
        except OSError:
            print(f'Failed to open path {profile_path}')
            raise
    
    pictures = defaultdict(open_picture) # open_picture需要一个参数，没法处理
    handle = pictures[path]
    handle.seek(0)
    image_data = handle.read()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\SAT\AppData\Local\Temp\ipykernel_40984\1101936220.py", line 18, in <module>
    handle = pictures[path]
TypeError: open_picture() missing 1 required positional argument: 'profile_path'


In [28]:
path = 'account_9090.csv'

with open(path, 'wb') as f:
    f.write(b'image data here 9090')

def open_picture(profile_path):
    try:
        return open(profile_path, 'a+b')
    except OSError:
        print(f'Failed to open path {profile_path}')
        raise

class Pictures(dict):
    def __missing__(self, key): # 当key不存在是，调用missing方法
        value = open_picture(key)
        self[key] = value
        return value

pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()
print(pictures)
print(image_data)

{'account_9090.csv': <_io.BufferedRandom name='account_9090.csv'>}
b'mage data here 9090'
